# Deep Learning

we do not need to do feature engineering manually, it is done by the model itself.

# Convolutional Neural Networks

In [ ]:
contrains

- you need huge data(100k samples - tabular data)
- unstructured dataset(images, videos, audios, etc - greater than 50k samples)

In [ ]:
Artifical Neural Network

1000x1000x1000 = 1B

In [ ]:
## real world applications

- image classification
- facial recognition -> DCNN(Deep Convolutional Neural Networks)
- object detection
- medical image analysis - Xrays, MRIs
- video analytics - action recognition
- NLP - text classification, language translation, sentiment analysis(SpaCy, NER)

# Convolution

In [ ]:
- convolution is a mathematical operation
- the fundamental part of CNN is a filter(matrix)

1*1 + 6*0 + 9*(-1) + 2*1 ,.... = 18


<img src='c1.png' />

<br />

<img src='c2.png' />

<br />

<img src='c4.png' />

<br />

<img src='c5.png' />

## what about colored image?

<img src='c6.png' />

In [ ]:
6x6x3 * 3x3x3 = 4x4x3

3 is representing RGB - Red, Green, Blue

# Padding

<img src='c7.png' />


In [ ]:
in the convolutional operation the corner pixel will be used only once as compared to other pixels

<img src='c8.png' />

# Apply padding

<img src='c9.png' />

# results

<img src='c10.png' />


# Stride

<img src='c11.png' />

In [ ]:
the above example - the stride is = 1

# Pooling

<img src='c12.png' />

# But Why

<img src='c13.png' />

In [ ]:
pooling is used to reduce the image size and keep the features intact.

where we are going to apply max pooling?
- after convolutional layer

best part about max pooling
- no parameters(weights & bias) that means no training

# Fully Connected Layer

<img src='c14.png' />

In [ ]:
FC layer -> dense network of neurons

- applied after convolutional and max pooling layers
- classify the output
- associate features to a particular label


# CNN Architecture

<img src='c15.png' />

In [ ]:
Activation Function - help the model classify the label

ReLU - Rectified Linear Unit

- non-linearity to the model
- the network is going to learn complex patterns and representations
- computational efficiency

<img src='c16.png' />

# Resources and Code


https://cs231n.github.io/convolutional-networks


Code:
https://blog.paperspace.com/writing-cnns-from-scratch-in-pytorch/


ReLU vs Sigmoid

https://wandb.ai/ayush-thakur/dl-question-bank/reports/ReLU-vs-Sigmoid-Function-in-Deep-Neural-Networks--VmlldzoyMDk0MzI




Dataset: 
https://www.cs.toronto.edu/~kriz/cifar.html

# Day 32 - CNN code

## Code

CNN with Cifar-10 DataSet - https://www.cs.toronto.edu/~kriz/cifar.html

In [ ]:
PyTorch

- Deep Learning framework

int(data structure in python) -> 4 bytes = 32 bits

ndarray -> numpy

in pytorch -> tensors(data strctures) ->(numerical values in matrix)

In [1]:
# all required imports

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# variables
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# where do you want to run your model? on GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# dataset

all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                    transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                         std=[0.2023, 0.1994, 0.2010])
                                    ])

train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                            train=True,
                                            transform=all_transforms,
                                            download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                           train=False,
                                           transform=all_transforms,
                                           download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

100%|██████████| 170498071/170498071 [01:10<00:00, 2412424.80it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# CNN

class ConvNeuralNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(1600,128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128,num_classes)
    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
        
        out = out.reshape(out.size(0), -1) # flatten - to help the network run fast(1 dimensional)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        
        return out

In [4]:
# hyperparameters

model = ConvNeuralNet(num_classes)

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, 
                            weight_decay=0.005, momentum=0.9)

total_step = len(train_loader)

In [5]:
# training

for epoch in range(num_epochs):
    
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass and optimizer
        optimizer.zero_grad()  #set the gradient to zero before every update
        loss.backward()        # calculate the new gradient
        optimizer.step()       # update the weights
    
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/20], Loss: 1.8332
Epoch [2/20], Loss: 1.4459
Epoch [3/20], Loss: 1.0226
Epoch [4/20], Loss: 0.6811
Epoch [5/20], Loss: 1.1840
Epoch [6/20], Loss: 1.2191
Epoch [7/20], Loss: 1.3997
Epoch [8/20], Loss: 0.8965
Epoch [9/20], Loss: 0.6669
Epoch [10/20], Loss: 1.5576
Epoch [11/20], Loss: 0.7188
Epoch [12/20], Loss: 0.6552
Epoch [13/20], Loss: 0.6240
Epoch [14/20], Loss: 1.1015
Epoch [15/20], Loss: 0.6861
Epoch [16/20], Loss: 0.8026
Epoch [17/20], Loss: 0.7967
Epoch [18/20], Loss: 0.6490
Epoch [19/20], Loss: 0.5523
Epoch [20/20], Loss: 0.7765


In [6]:
# training dataset 

with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print("Accuracy on {} train images = {}%".format(50000, 100*correct/total))

Accuracy on 50000 train images = 83.048%


In [7]:
# testing

with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print("Accuracy on {} testing images = {}%".format(10000, 100*correct/total))

Accuracy on 10000 testing images = 68.8%


In [11]:
# testing

with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data,1)
        print(predicted)

In [14]:
predicted

tensor([4, 2, 9, 1, 8, 5, 6, 9, 0, 6, 5, 6, 5, 7, 4, 6])

In [15]:
labels

tensor([0, 2, 9, 1, 0, 5, 6, 9, 0, 4, 7, 6, 7, 5, 7, 6])

In [17]:
import matplotlib.pyplot as plt